# Задача: выполнить какуб-нибудь задачу используя PyTorch


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
import torch.utils.data as data
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm

# Импортируем все библиотеки

In [2]:
!kaggle datasets download -d msambare/fer2013 # Скачаем датасет

 76% 46.0M/60.3M [00:00<00:00, 157MB/s]
100% 60.3M/60.3M [00:00<00:00, 162MB/s]


In [3]:
import zipfile

zip_file_path = '/content/fer2013.zip'

extract_to_path = '/content'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
# Разархивируем

In [6]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
]
)
# Функция предобработки изображений

In [7]:
train_folder_path = '/content/train'

train_dataset = datasets.ImageFolder(root=train_folder_path, transform=transform) # Загрузим и предобработаем тренировочную выборку

In [42]:
classes = train_dataset.classes
print(classes)
class_counts = train_dataset.class_to_idx
print(class_counts)
i2c = {0 : 'angry', 1 : 'disgust', 2 : 'fear', 3 : 'happy', 4 : 'neutral', 5: 'sad', 6:'surprise'}

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [9]:
test_folder_path = '/content/test'

test_dataset = datasets.ImageFolder(root=test_folder_path, transform=transform)
#Загрузим и предобработаем тестовую выборку

In [11]:
train_dl = data.DataLoader(train_dataset, batch_size=32, shuffle=False)
test_dl = data.DataLoader(test_dataset, batch_size=32, shuffle=False)
# Создадим даталоадеры для обучения модели

In [54]:
model = models.resnet50(pretrained=True) # Загрузим предобученную ResNet50

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth

  0%|          | 0.00/97.8M [00:00<?, ?B/s]
  8%|▊         | 7.40M/97.8M [00:00<00:01, 77.5MB/s]
 23%|██▎       | 22.5M/97.8M [00:00<00:00, 125MB/s] 
 40%|████      | 39.5M/97.8M [00:00<00:00, 149MB/s]
 55%|█████▍    | 53.7M/97.8M [00:00<00:00

In [15]:
for param in model.parameters():
    param.requires_grad = False
# Заморозим слои

In [25]:
batch_size = 32
num_epochs = 5
num_classes = len(classes)
learning_rate = 0.001
# Гиперпараметры

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Создадим device

In [55]:
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
# Заменим голову модели и загрузим на гпу саму модель

In [56]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Создадим оптимизатор

In [57]:
scheduler = OneCycleLR(optimizer, max_lr=0.01, total_steps=len(train_dl)*num_epochs)
# Скедулер

In [58]:
loss_fn = nn.CrossEntropyLoss()
# Лосс-функция

In [60]:
# Процесс обучения

for epoch in range(num_epochs):
    running_loss = 0.0
    all = 0
    correct = 0
    model.train()

    pbar = tqdm(total=len(train_dl))

    for images, y in train_dl:
        images = images.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        y_pred = model(images)
        loss = loss_fn(y_pred, y)
        # Посчитали лосс
        loss.backward()
        optimizer.step()

        # Обновили веса

        running_loss += loss.item()
        _, predicted = torch.max(y_pred.data, 1)
        all += y_pred.size(0)
        correct += (predicted == y).sum().item()
        # Подсчет метрики
        pbar.update(1) # Обновление прогресс бара

    pbar.close() # Закрытие прогресс бара прошедшей эпохи

    epoch_loss = running_loss / len(train_dl) # Потеря
    epoch_accuracy = 100 * correct / all # Точность

    print(f'Эпоха номер: {epoch+1}/{num_epochs}: Ошибка: {epoch_loss:.4f}, Метрика accuracy: {epoch_accuracy:.2f}%')
    # Вывод

100%|██████████| 898/898 [05:30<00:00,  2.72it/s]


Эпоха номер: 1/5: Ошибка: 0.9442, Метрика accuracy: 80.26%


  4%|▎         | 32/898 [00:11<05:11,  2.78it/s]

KeyboardInterrupt: 

# Здесь я пытался развернуть модель в GUI Gradio, но возникала ошибка которую я не знаю как исправить
#Сама ошибка возникает при проходе изображения через transform

In [40]:
!pip install gradio
import gradio as gr

In [ ]:
model.eval() # Перевели модель в режим тестирования

In [105]:
def classify_image(image):
    image = transform(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        class_index = predicted.item()

    return class_index

def classify_interface(image):
    class_index = classify_image(image)
    return f"Class Index: {class_index}"

# Функция для связи GUI с Моделью

In [106]:
iface = gr.Interface(
    fn=classify_interface,
    inputs="image",
    outputs="text",
    title="Модель классификации изображений",
    description="Загрузите изображение и модель предскажет его класс."
)

# Интерфейс и его запуск

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5a6bd915054b04859d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [108]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dl:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy модели на тестовой выборке: {accuracy:.2f}%")

Accuracy модели на тестовой выборке: 11.58%


Что-то не так с точностью не тестовой выборке, на тренировке она была 80%, а здесь 11%